<a href="https://colab.research.google.com/github/nguyentienlocsmile/demo/blob/main/Transfer_Learning_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Speaker: Ba Ngoc - Founder @ProtonX @VietAI Hanoi

- Bài tập phân loại ảnh sử dụng Transfer Learning - Bài tập có 4 phần cần hoàn thành

In [ ]:
#@title 1. Tải dữ liệu
train_folder = './data/cats_and_dogs_filtered/train'
valid_folder = './data/cats_and_dogs_filtered/validation'

!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O ./cats_and_dogs_filtered.zip

# Giải nén dữ liệu
main_folder = './data'
!unzip -q cats_and_dogs_filtered.zip -d $main_folder

In [ ]:
# Sử dụng colab để review ảnh

In [ ]:
#@title 2. Tải Pretrained model - Inception V3

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O ./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [ ]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model


# 1. Sử dụng kiến trúc của mô hình
from tensorflow.keras.applications.inception_v3 import InceptionV3

# 2. Khởi tạo mô hình
pre_trained_model = #### TODO 1: Sử dụng mô hình InceptionV3

local_weights_file = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# 3. Load tham số đã được train vào mô hình
pre_trained_model.load_weights(local_weights_file)

In [ ]:
# 4. Đóng băng (freeze) các lớp, trường hợp này chỉ sử dụng các tham số này 
# để inference (dự đoán) mà không sử dụng để training 

# 5. Câu hỏi (Có thể sử dụng các tham số này để tiếp tục huấn luyên không - Câu trả lời: Có)
for layer in pre_trained_model.layers:
  #### TODO 2: Cài đặt đóng băng các layer

In [ ]:
# 6. Lấy layer ta muốn gắn thêm lớp phân loại theo bài toán của mình

#### TODO 3: Sử dụng lại mô hình đến layer mixed7


last_output = last_layer.output

In [ ]:
last_output

# 5. Xây dựng mô hình

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# 7. Duỗi features map thu được và đựa vào mạng nơ ron để phân loại
x = layers.Flatten()(last_output)
# 8. Thêm một lớp Dense
x = layers.Dense(1024, activation='relu')(x)
# 9. Thêm một lớp Dropout để cải thiện tính tổng quát của mô hình - tránh overfiting
x = layers.Dropout(0.2)(x)                  
# 10. Sử dụng activation sigmoid để phân loại 2 lớp
x = layers.Dense(1, activation='sigmoid')(x)           

# 11. Gắn mô hình phân loại của chúng ta vào pre trained model
model = Model( pre_trained_model.input, x) 

# 12. Bổ sung thuật toán trainining. RMSProp có khả năng điều chỉnh tốc độc học dựa vào độ lớn của đạo hàm

#### TODO 4: Compile thuật toán
              

In [ ]:
#@title Hiển thị mô hình
import tensorflow as tf
tf.keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
)

In [ ]:
#@title Tăng cường data
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 13. Bổ sung tăng cường data cho tập train
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# 13. Không bổ sung tăng cường data cho tập validation
validation_datagen = ImageDataGenerator( rescale = 1.0/255. )

# 14. Tiến hành load ảnh từ thư mục với tập train
train_generator = train_datagen.flow_from_directory(train_folder,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# 15. Tiến hành load ảnh từ thư mục với tập validation
validation_generator =  validation_datagen.flow_from_directory( valid_folder,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

In [ ]:
train_generator.class_indices

In [ ]:
# 16. Tiến hành training 

#### TODO 5: Tiến hành training

# 6. Tải ảnh của bạn và tiến hành dự đoán

In [ ]:
#@title Tải ảnh
from google.colab import files
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
  image = plt.imread(fn)
  plt.imshow(image)
  

  image = Image.open(fn)
  # img = Image.fromarray(arr)
  image = image.resize(size=(150, 150))

  plt.imshow(image)
  data = np.asarray(image)
  data=np.expand_dims(data, 0)
  x = data / 255.0
  fi = model.predict(x)
  print(fi)
  print('Dog with output: {}'.format(fi[0])) if fi >= 0.5 else print('Cat with output: {}'.format(fi[0]))